In [ ]:
import zipfile as zf
files = zf.ZipFile("datasets.zip", 'r')
files.extractall('.')
files.close()

In [ ]:
from pathlib import Path

zip_path = Path("datasets.zip")
zf.ZipFile(zip_path,'r')

# Loading categorical text data 

In [ ]:
from sklearn.datasets import load_files
from pathlib import Path

folder_path = Path('./datasets/text_contents')

load_files_test = load_files(folder_path)

In [ ]:
load_files_test

In [ ]:
type(load_files_test)

In [ ]:
load_files_test.target_names

In [ ]:
load_files_test.data

In [ ]:
load_files_test.filenames

# Bag of Words Feature Extraction

Reference : https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [ ]:
# import the library tha will help us create the bag of words representation of a document. 

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
vectorizer.get_params

In [ ]:
vectorizer.build_analyzer()("This is a python DS class!!, is it ??")

In [ ]:
type(vectorizer)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
## Load our three documents and use fit to create the list of our bag of words
vectorizer.fit(load_files_test.data)

In [ ]:
features = vectorizer.get_feature_names()

In [ ]:
type(features)

In [ ]:
len(features)

In [ ]:
features

In [ ]:
bag_of_words = vectorizer.transform(load_files_test.data)

In [ ]:
type(load_files_test.data[0])

In [ ]:
type(bag_of_words)

### Compressed Sparse Row Matrix

https://en.wikipedia.org/wiki/Sparse_matrix 

In [ ]:
bag_of_words.shape

In [ ]:
bag_of_words[0]

In [ ]:
type(bag_of_words[0])

In [ ]:
bag_of_words[0].shape

In [ ]:
bag_of_words[0].size

In [ ]:
print(bag_of_words[0])

In [ ]:
bag_of_words[0].toarray()

In [ ]:
type(vectorizer.vocabulary_)

In [ ]:
vectorizer.vocabulary_

In [ ]:
len(vectorizer.vocabulary_)

## Tf-Idf

https://en.wikipedia.org/wiki/Tf%E2%80%93idf

# Pipeline & Grid Search

In [ ]:
import pandas as pd 
import string

from pprint import pprint

from time import time

import os

## Load the data

For this example we use the SMS spam collection data set from UCI machine learning lab.

https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [ ]:
file_path = os.path.relpath('./datasets/SMSSpamCollection.csv')

In [ ]:
file_path

In [ ]:
sms_spam_collection_df = pd.read_csv(file_path,sep='\t',names=['target','message'])

In [ ]:
type(sms_spam_collection_df)

## Explore the data

In [ ]:
sms_spam_collection_df.head()

In [ ]:
sms_spam_collection_df.describe()

## Create train and test splits

In [ ]:
from sklearn.model_selection import train_test_split

message_train,message_test,target_train,target_test = train_test_split(sms_spam_collection_df['message'],sms_spam_collection_df['target'])

In [ ]:
print(message_train.shape,message_test.shape)

In [ ]:
type(target_train)

- [ ] **30 sec: Explore the train and test splits**

## Build our Model 

**The model in this case requires a pipeline, because the input is text and it cannot be directly fed into our model.**

The pipeline will consist of a CountVectorizer --> TfidfTrasnformer --> Classifier. 

The classifier we will choose for this exercise is SGD Classifer

In [ ]:
# import vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

#import SGD classifier 

from sklearn.linear_model import SGDClassifier


#### Build the desired pipeline

In [ ]:
# create our data processing pipeline

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
                     ('vectorizer',CountVectorizer()),
                     ('tf-idf',TfidfTransformer()),
                     ('sgd_classifier',SGDClassifier()),
                    ])



## Train the Model 

### Hyper parameter search using GridSearch 

https://scikit-learn.org/stable/modules/grid_search.html#grid-search

https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py

In [ ]:
#import grid serach 

from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline

In [ ]:
pipeline.steps

In [ ]:
type(pipeline.steps[1][1])

In [ ]:
tfidf_transformer = pipeline.steps[1][1]

In [ ]:
## Lets check out all the hyper parameters available 
tfidf_transformer.get_params()

In [ ]:
## We choose the use_idf parameter to tune 
grid_search_params = {'tf-idf__use_idf':(True,False),
                     'sgd_classifier__max_iter': (10, 50, 80,1000),
                      'vectorizer__max_features':(None,10,50,1000)
                     }

In [ ]:
grid_search = GridSearchCV(pipeline,grid_search_params,n_jobs=-1,verbose=1,cv=5)

In [ ]:
type(grid_search)

In [ ]:
print("started grid search")

print("parameters are :")

pprint(grid_search_params)

In [ ]:
start_time = time()

grid_search.fit(message_train,target_train)
print('doine in %f secdons' %(time()-start_time))

In [ ]:
grid_search_params

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

## Test 

In [ ]:
pred_target = grid_search.predict(message_test)

In [ ]:
pred_target

In [ ]:
result = (pred_target == target_test)

In [ ]:
result.values

In [ ]:
## find the accurary from this 

accuracy = (result.value_counts()[True] / result.count())

accuracy

In [ ]:
result.value_counts()[True]

In [ ]:
result.value_counts()[False]